In [2]:
# HACK: use project root as the working directory 
from pathlib import Path

while Path.cwd().name != 'language-model-toxicity':
    %cd ..

/homes/gws/sgehman/language-model-toxicity


In [3]:
import logging
from typing import List

import dask
from joblib import Memory, Parallel, delayed, dump
from lsh import cache, minhash
import numpy as np
from itertools import chain, islice

from utils.constants import DATA_DIR, OUTPUT_DIR
from utils.webtext import load_meta, delayed_corpus, split_docs

# Create joblib memory
mem = Memory(OUTPUT_DIR / 'cache' / 'webtext_overlap')

In [4]:
cached_meta = mem.cache(load_meta)

wt_meta = cached_meta(DATA_DIR / 'webtext')
wt_files = wt_meta[0]

In [5]:
wt = delayed_corpus(wt_meta)

In [6]:
wt = wt.compute()

In [8]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

In [11]:
detokenized_wt = Parallel(n_jobs=64, verbose=1)(
    delayed(tokenizer.decode)(doc, clean_up_tokenization_spaces=False)
    for doc in wt
)

[Parallel(n_jobs=64)]: Using backend LokyBackend with 64 concurrent workers.


KeyboardInterrupt: 

In [13]:
import multiprocessing as mp

In [17]:
def detokenize(doc):
    return tokenizer.decode(doc, clean_up_tokenization_spaces=False)

In [19]:
from tqdm.auto import tqdm

In [22]:
with mp.Pool(processes=96) as pool:
    detokenized_docs = list(tqdm(pool.imap(detokenize, wt, chunksize=10_000)))

In [23]:
len(detokenized_docs)

8282020

In [24]:
detokenized_docs[0]

"The United Arab Emirates has seen yet another record-breaking solar power tariff bid. Abu Dhabi received the lowest-ever bid for a solar PV project at a shocking 2.42¢/kWh, taking back the title of cheapest solar power project from Chile.\n\nThe record-breaking bid has been submitted by a consortium of Chinese module manufacturer JinkoSolar and Japanese developer Marubeni.\n\nAbu Dhabi Electricity and Water Authority received a total of 6 bids for the proposed 350 MW solar PV project planned to be built in the town of Swaihan, Abu Dhabi. Out of 6 bids, the lowest ever bid of 2.42¢/kWh has been submitted by the JinkoSolar–Marubeni consortium. The results of the tender are not out yet, as authorities will now evaluate the proposals for technical and economic viability.\n\nThe current bid of 2.42¢/kWh is the lowest so far globally, and by quite a bit — it is shockingly low. This bid is 20% lower than the previous record bid of 2.91¢/kWh submitted at an auction in Chile last month.\n\nThe

In [25]:
dump(detokenized_docs, OUTPUT_DIR / 'detokenized_webtext.joblib')

['output/detokenized_webtext.joblib']

In [29]:
shard_len = 414101
assert len(detokenized_docs) % shard_len == 0

In [30]:
shard_i = 0
for i in range(0, len(detokenized_docs), shard_len):
    dump(detokenized_docs[i: i + shard_len], OUTPUT_DIR / 'detokenized_webtext' / f"webtext_{shard_i:02d}.joblib")
    shard_i += 1

In [31]:
detokenized_docs[-1]

'"Bold Berry" Print on the Nike Roshe One\n\n4.68 / 5 52 VOTES\n\nThe Nike Roshe Run is now about to enter its 3rd straight summer, and after the recent rename to the Nike Roshe One, we\'re patiently waiting to see if Nike has bigger plans. In the meantime, the Swoosh is doing what it does every summer to the silhouette – updating it with a number of festive prints and patterns while seasonal appropriate materials like mesh and nylon come together to make the shoe even more comfortable. The latest aesthetic features a clean white throughout while splashes of what the brand is calling \'Bold Berry\' arrives in the form of cartoonish flowers on the side panel. The Swoosh is camouflaged in a matching white so that the array of electric blue, purple and green attracts the most attention. Pairs have already arrived at international shops so expect this Roshe Run to be available just in time for summer.\n\nSource: shelflife\n\nNike WMNS Roshe Run\n\nColor: White/Bold Berry\n\nStyle Code: 599